In [1]:
import pandas as pd
import janitor
from io import StringIO
from pandas import Timestamp
import numpy as np

In [2]:
data = """ A    B   C      D          E
    0  0  0.0   r  False   1970-01-01
    1  0  0.0   r  False   1970-01-01"""

df = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python')
df

,A,B,C,D,E
0,0,0.0,r,False,1970-01-01
1,0,0.0,r,False,1970-01-01


In [3]:
df['E'] = pd.to_datetime(df['E'])
df

,A,B,C,D,E
0,0,0.0,r,False,1970-01-01
1,0,0.0,r,False,1970-01-01


In [4]:
data = """Integers  Numeric  Floats  Strings  Booleans      Dates   Dates_Right
    0         0      0.0     0.0      r       False   1970-01-01  1970-01-01
    1         0     -1.0     0.0      r       False   1970-01-01  1970-01-01"""

right = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python')
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,0.0,r,False,1970-01-01,1970-01-01
1,0,-1.0,0.0,r,False,1970-01-01,1970-01-01


In [5]:
right['Dates'] = pd.to_datetime(right['Dates'])
right['Dates_Right'] = pd.to_datetime(right['Dates_Right'])
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,0.0,r,False,1970-01-01,1970-01-01
1,0,-1.0,0.0,r,False,1970-01-01,1970-01-01


In [6]:
middle, left_on, right_on = ("B", "Numeric", "Floats")
l = df.conditional_join(
        right,
        (middle, left_on, ">="),
        (middle, right_on, "<="),
        how="inner",
        sort_by_appearance=True,
    )#.filter([left_on, middle, right_on])

l

A    B  C      D          E
0 0  0  0.0  r  False 1970-01-01
  1  0  0.0  r  False 1970-01-01
1 0  0  0.0  r  False 1970-01-01
  1  0  0.0  r  False 1970-01-01

In [41]:
l[0]

,A,B,C,D,E
0,0,0.0,r,False,1970-01-01
1,0,0.0,r,False,1970-01-01


In [42]:
l[1]

,A,B,C,D,E
0,0,0.0,r,False,1970-01-01
1,0,0.0,r,False,1970-01-01


In [43]:
expected = (
        df.assign(t=1)
        .merge(right.assign(t=1), on="t")
        .query(f"{left_on} <= {middle} <= {right_on}")
        #.reset_index(drop=True)
    )

expected.filter([left_on, middle, right_on])

,Numeric,B,Floats
0,0.0,0.0,0.0
1,-1.0,0.0,0.0
2,0.0,0.0,0.0
3,-1.0,0.0,0.0


In [44]:
right.to_dict('records')

[{'Integers': 0,
  'Numeric': 0.0,
  'Floats': 0.0,
  'Strings': 'r',
  'Booleans': False,
  'Dates': Timestamp('1970-01-01 00:00:00'),
  'Dates_Right': Timestamp('1970-01-01 00:00:00')},
 {'Integers': 0,
  'Numeric': -1.0,
  'Floats': 0.0,
  'Strings': 'r',
  'Booleans': False,
  'Dates': Timestamp('1970-01-01 00:00:00'),
  'Dates_Right': Timestamp('1970-01-01 00:00:00')}]

In [45]:
In [11]: df = pd.DataFrame([{'A': 0,
    ...:   'B': 0.0,
    ...:   'C': 'r',
    ...:   'D': False,
    ...:   'E': Timestamp('1970-01-01 00:00:00')},
    ...:  {'A': 0,
    ...:   'B': 0.0,
    ...:   'C': 'r',
    ...:   'D': False,
    ...:   'E': Timestamp('1970-01-01 00:00:00')}])

In [12]: right = pd.DataFrame([{'Integers': 0,
    ...:   'Numeric': 0.0,
    ...:   'Floats': 1,
    ...:   'Strings': 'r',
    ...:   'Booleans': False,
    ...:   'Dates': Timestamp('1970-01-01 00:00:00.000000001'),
    ...:   'Dates_Right': Timestamp('1970-01-01 00:00:00')},
    ...:  {'Integers': 0,
    ...:   'Numeric': 0.0,
    ...:   'Floats': 1,
    ...:   'Strings': 'r',
    ...:   'Booleans': True,
    ...:   'Dates': Timestamp('1970-01-01 00:00:00'),
    ...:   'Dates_Right': Timestamp('1970-01-01 00:00:00')}])

In [46]:
df.conditional_join(
        right, ('E', 'Dates', ">"), how="right", sort_by_appearance=True
    )

0    Empty DataFrame
Columns: [A, B, C, D, E]
Index...
1    Empty DataFrame
Columns: [A, B, C, D, E]
Index...
dtype: object

In [47]:
df

,A,B,C,D,E
0,0,0.0,r,False,1970-01-01
1,0,0.0,r,False,1970-01-01


In [48]:
right

,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,0,0.0,1,r,False,1970-01-01 00:00:00.000000001,1970-01-01
1,0,0.0,1,r,True,1970-01-01 00:00:00.000000000,1970-01-01


In [49]:
left_on, right_on = ["E", "Dates"]
expected = (
    df.assign(t=1)
    .merge(right.assign(t=1, index=np.arange(len(right))), on="t")
    .query(f"{left_on} > {right_on}")
)
expected = expected.set_index("index")
expected.index.name = None
expected = (
    expected.filter(df.columns)
    .join(right, how="right", sort=False)
    .reset_index(drop=True)
)

expected

,A,B,C,D,E,Integers,Numeric,Floats,Strings,Booleans,Dates,Dates_Right
0,NaN,NaN,NaN,NaN,NaT,0,0.0,1,r,False,1970-01-01 00:00:00.000000001,1970-01-01
1,NaN,NaN,NaN,NaN,NaT,0,0.0,1,r,True,1970-01-01 00:00:00.000000000,1970-01-01


In [12]:
data = """  timestamp              A          B
0   2016-05-14 10:54:33    0.020228   0.026572
1   2016-05-14 10:54:34    0.057780   0.175499
2   2016-05-14 10:54:35    0.098808   0.620986
3   2016-05-14 10:54:36    0.158789   1.014819
4   2016-05-14 10:54:39    0.038129   2.384590"""

df_1 = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python', parse_dates=['timestamp'])
df_1

,timestamp,A,B
0,2016-05-14 10:54:33,0.020228,0.026572
1,2016-05-14 10:54:34,0.057780,0.175499
2,2016-05-14 10:54:35,0.098808,0.620986
3,2016-05-14 10:54:36,0.158789,1.014819
4,2016-05-14 10:54:39,0.038129,2.384590


In [13]:
data = """  start                end                  event    
0   2016-05-14 10:54:31  2016-05-14 10:54:33  E1
1   2016-05-14 10:54:34  2016-05-14 10:54:37  E2
2   2016-05-14 10:54:38  2016-05-14 10:54:42  E3"""

df_2 = pd.read_csv(StringIO(data), sep='\s{2,}', engine='python', parse_dates=['start', 'end'])
df_2

,start,end,event
0,2016-05-14 10:54:31,2016-05-14 10:54:33,E1
1,2016-05-14 10:54:34,2016-05-14 10:54:37,E2
2,2016-05-14 10:54:38,2016-05-14 10:54:42,E3


In [15]:
event_num = len(df_2.event)

def get_event(t, df_2):    
    event_idx = ((t >= df_2.start) & (t <= df_2.end)).dot(np.arange(event_num))
    return event_idx
    return df_2.event[event_idx]

df_1.timestamp.transform(get_event, df_2 = df_2)

# df_1["event"] = df_1.timestamp.transform(get_event)

0    0
1    1
2    1
3    1
4    2
Name: timestamp, dtype: int64

In [11]:
df_1

,timestamp,A,B,event
0,2016-05-14 10:54:33,0.020228,0.026572,E1
1,2016-05-14 10:54:34,0.057780,0.175499,E2
2,2016-05-14 10:54:35,0.098808,0.620986,E2
3,2016-05-14 10:54:36,0.158789,1.014819,E2
4,2016-05-14 10:54:39,0.038129,2.384590,E3
